In [9]:
import sqlite3
import sqlalchemy
import pandas as pd

In [ ]:
try:
    conn = sqlite3.connect("wjazzd.db")
except Exception as e:
    print(e)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Table Name : {cursor.fetchall()}")

df = pd.read_sql_query('''
                       SELECT melody.melid, 
                           melody.pitch, 
                           melody.onset as start,
                           melody.onset + melody.duration as end,
                           melody.duration, 
                           solo_info.instrument,
                           solo_info.key,
                           solo_info.style,
                           solo_info.avgtempo as tempo,
                           solo_info.rhythmfeel as feel,
                           beats.beat as beat
                        FROM melody
                        JOIN solo_info
                        ON melody.melid = solo_info.melid
                       ''', conn)
conn.close()

Table Name : [('melody',), ('sqlite_sequence',), ('beats',), ('sections',), ('melody_type',), ('transcription_info',), ('solo_info',), ('record_info',), ('track_info',), ('composition_info',), ('esac_info',), ('popsong_info',), ('db_info',)]


In [15]:
df


,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,1,1,9.171882,-1,1,,,I1,42.0,0
1,2,1,9.488254,-1,2,,,,42.0,0
2,3,1,9.779955,-1,3,,,,40.0,0
3,4,1,10.052608,-1,4,,,,40.0,0
4,5,1,10.339796,0,1,,Bb6,,50.0,0
...,...,...,...,...,...,...,...,...,...,...
132324,132325,456,64.713311,55,2,,,,NaN,2
132325,132326,456,65.005896,55,3,,,,NaN,2
132326,132327,456,65.289524,55,4,,,,NaN,2
132327,132328,456,65.573152,56,1,,NC,,NaN,2


In [ ]:
df.to_csv('pitches_w_instruments.csv')

In [ ]:
try:
    conn = sqlite3.connect("wjazzd.db")
except Exception as e:
    print(e)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Table Name : {cursor.fetchall()}")

df = pd.read_sql_query('''
                       SELECT *
                       FROM beats
                       ''', conn)
conn.close()

In [ ]:
import my_functions
import importlib
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional

import my_functions
import importlib
importlib.reload(my_functions)

In [ ]:
df = my_functions.extract_notes()

In [ ]:
df.head()
df['step'] = df['start'] - df['start'].shift(1)

# Fix up the first one cause of the wacky shift problem
df.loc[0,"step"] = 0

In [ ]:
def plot_distributions(notes: pd.DataFrame, drop_percentile=2.5):
  plt.figure(figsize=[15, 5])
  plt.subplot(1, 3, 1)
  sns.histplot(notes, x="pitch", bins=20)

  plt.subplot(1, 3, 2)
  max_step = np.percentile(notes['step'], 100 - drop_percentile)
  sns.histplot(notes, x="step", bins=np.linspace(0, max_step, 21))
  
  plt.subplot(1, 3, 3)
  max_duration = np.percentile(notes['duration'], 100 - drop_percentile)
  sns.histplot(notes, x="duration", bins=np.linspace(0, max_duration, 21))

In [ ]:
plot_distributions(df[df['instrument']=='as'])

In [ ]:
[df['instrument'] == 'cl']